Verify the sobol sequence as used in the CosmoGrid

In [1]:
import os, h5py
import numpy as np
from sobol_seq import i4_sobol
from scipy.stats.qmc import Sobol
from icecream import ic

from msfm.utils import cosmogrid
from msfm.utils.input_output import read_yaml

In [2]:
cosmo_params_info = cosmogrid.get_cosmo_params_info("../../data/CosmoGridV1_original_metainfo.h5", "grid")

### verify with respect to the CosmoGrid
Differences in `Ob` are expected due to bugs in concept

In [30]:
# order like the Sobol sequence
labels = ["Om", "s8", "H0", "Ob", "ns", "w0"]
cosmo_grid_priors = np.array(
    [
        [0.1, 0.5],
        [0.4, 1.4],
        [64.0, 82.0],
        [0.03, 0.06],
        [0.87, 1.07],
        [-2.0, -0.333],
    ], dtype=np.float32
)

# TODO There's a bug with respect to h!

In [33]:
for i in range(len(cosmo_params_info)):
    cosmo_params = [cosmo_params_info[label][i] for label in labels]
    cosmo_params = np.array(cosmo_params, dtype=np.float32)

    i_sobol = cosmo_params_info["sobol_index"][i]
    sobol_point, _ = i4_sobol(6, i_sobol)
    sobol_params = sobol_point*np.squeeze(np.diff(cosmo_grid_priors)) + cosmo_grid_priors[:,0]
    sobol_params = sobol_params.astype(np.float32)

    if i_sobol == 60611:
        print(cosmo_params[2])
        print(sobol_params[2], "\n")

# in /home/ipa/refreg/data/data_products/CosmoGrid/raw/grid/cosmo_060611/run_0/params.yml:
# 68.8507385254

68.85074
73.063446 



In [43]:
counter = 0
for i in range(len(cosmo_params_info)):
    cosmo_params = [cosmo_params_info[label][i] for label in labels]
    cosmo_params = np.array(cosmo_params, dtype=np.float32)

    i_sobol = cosmo_params_info["sobol_index"][i]
    sobol_point, _ = i4_sobol(6, i_sobol)
    sobol_params = sobol_point*np.squeeze(np.diff(cosmo_grid_priors)) + cosmo_grid_priors[:,0]
    sobol_params = sobol_params.astype(np.float32)

    print("\n", i_sobol)
    # print(np.isclose(sobol_params, cosmo_params, rtol=1e-3, atol=1e-5), "\n")
    
    # Ob
    print(cosmo_params[3])
    print(sobol_params[3])

    # H0
    # print(cosmo_params[2])
    # print(sobol_params[2], "\n")

    if not np.isclose(cosmo_params[2], sobol_params[2], rtol=1e-3, atol=1e-5):
        counter += 1

print(counter)


 1
0.045
0.044999998

 2
0.0525
0.037499998

 3
0.0375
0.0525

 4
0.04875
0.033749998

 6
0.04125
0.041249998

 10
0.046875
0.058125

 11
0.031875
0.043125

 12
0.058125
0.046875

 14
0.035625
0.054375

 19
0.0478125
0.049687497

 20
0.0440625
0.0384375

 22
0.0515625
0.030937498

 23
0.0365625
0.045937497

 28
0.0346875
0.0590625

 30
0.0571875
0.0515625

 34
0.04078125
0.05484375

 39
0.04453125
0.04359375

 44
0.04265625
0.030468749

 47
0.03515625
0.052968748

 54
0.03421875
0.04828125

 55
0.04921875
0.033281248

 56
0.03234375
0.03890625

 58
0.05484375
0.03140625

 62
0.04359375
0.03515625

 63
0.05859375
0.05015625

 66
0.050859377
0.046640623

 70
0.032109376
0.050390624

 71
0.047109377
0.035390623

 78
0.037734374
0.033515625

 79
0.052734375
0.048515625

 82
0.042421874
0.056953125

 84
0.031171875
0.045703124

 87
0.038671874
0.038203124

 90
0.036796875
0.040078126

 91
0.051796876
0.055078123

 95
0.033046875
0.058828123

 97
0.042890623
0.055546872

 98
0.035390627
0.0

### single example

In [3]:
# from https://cosmo-gitlab.phys.ethz.ch/jafluri/cosmogrid_kids1000/-/blob/master/kids1000_analysis/constants.py#L13
priors = np.array([[ 0.1  ,  0.5  ],
                   [ 0.4  ,  1.4  ],
                   [ 0.64 ,  0.82 ],
                   [ 0.03 ,  0.06 ],
                   [ 0.87 ,  1.07 ],
                   [-2.   , -0.333],
          np.log10([1.0e12, 1.0e15]),
                   [-2.   ,  2.   ],
                   [-3.   ,  3.   ]])

# /home/ipa/refreg/data/data_products/CosmoGrid/raw/grid/cosmo_011062/run_0
sobol_seed = 11062

# https://cosmo-gitlab.phys.ethz.ch/jafluri/cosmogrid_kids1000/-/blob/master/kids1000_analysis/input_pipeline.py#L461
sobol_point, _ = i4_sobol(9, sobol_seed)
Om, s8, h, Ob, ns, w0, log10_Mc, nu, a_ia = (sobol_point*np.squeeze(np.diff(priors)) + priors[:,0])

ic(Om)
ic(s8)
ic(h)
ic(Ob)
ic(ns)
ic(w0)
ic(log10_Mc)
ic(nu)
ic(a_ia)

# content of params.yml
# As: 8.122750437129466e-10
# bary_Mc: 2943968138429.53
# bary_nu: 1.037841796875
# H0: 68.5758056641
# O_cdm: 0.336458858089
# O_nu: 0.0013725628089999858
# Ob: 0.045737915039
# Ol: -1.0
# Om: 0.383569335937
# m_nu: 0.02
# ns: 0.990397949219
# pkd_seed: 222536
# s8: 0.618078613281
# sobol_seed: 11062
# w0: -1.073199279785
# wa: 0.0

ic| Om: 0.3835693359375
ic| s8: 0.61807861328125
ic| h: 0.734427490234375
ic| Ob: 0.0376263427734375
ic| ns: 0.99039794921875
ic| w0: -1.0731992797851562
ic| log10_Mc: 12.46893310546875
ic| nu: 1.037841796875
ic| a_ia: 2.3565673828125


2.3565673828125

### comparison to scipy (not necessary)

In [6]:
print(Sobol(d=5, scramble=False).fast_forward(10).random())
sobol_point, _ = i4_sobol(5, 10)
print(sobol_point)

[[0.9375 0.0625 0.6875 0.1875 0.3125]]
[0.9375 0.0625 0.5625 0.9375 0.3125]


In [7]:
sobol_point, _ = i4_sobol(5, 10)
print(sobol_point)
sobol_point, _ = i4_sobol(10, 10)
print(sobol_point)


[0.9375 0.0625 0.5625 0.9375 0.3125]
[0.9375 0.0625 0.5625 0.9375 0.3125 0.4375 0.8125 0.6875 0.4375 0.8125]


# config

In [35]:
with h5py.File("../../data/CosmoGridV1_original_metainfo.h5") as f:
    meta_info = f["parameters/grid"][:]

In [36]:
conf_dir = "../../configs/config.yaml"
conf = read_yaml(conf_dir)

In [40]:
prior = np.array(conf["analysis"]["grid"]["prior"]["sobol"], dtype=np.float32)

In [41]:
prior

array([[ 1.00e-01,  5.00e-01],
       [ 4.00e-01,  1.40e+00],
       [ 6.40e+01,  8.20e+01],
       [ 3.00e-02,  6.00e-02],
       [ 8.70e-01,  1.07e+00],
       [-2.00e+00, -3.33e-01],
       [-3.00e+00,  3.00e+00]], dtype=float32)

In [42]:
cosmo_grid_priors.shape

(6, 2)